In [16]:
import pandas as pd
import ast


Realizamos la lectura del dataframe

In [146]:
rutamerge = r'C:\Users\Coder\Documents\PI_ML_OPS\Datos\mergefinal.parquet'
df_ml = pd.read_parquet(rutamerge)

In [147]:
df_ml.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17628 entries, 0 to 17627
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   user_id             17628 non-null  object 
 1   title               17628 non-null  object 
 2   developer           17628 non-null  object 
 3   genres              17628 non-null  object 
 4   release_date        17628 non-null  float64
 5   price               17628 non-null  float64
 6   playtime_forever    17628 non-null  int64  
 7   posted              17628 non-null  float64
 8   item_id             17628 non-null  int64  
 9   recommend           17628 non-null  bool   
 10  sentiment_analysis  17628 non-null  int64  
dtypes: bool(1), float64(3), int64(3), object(4)
memory usage: 1.4+ MB


In [148]:
df_ml

,user_id,title,developer,genres,release_date,price,playtime_forever,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,Counter-Strike,Valve,['Action'],2000.0,9.99,6,2011.0,1250,True,2
1,js41637,Counter-Strike,Valve,['Action'],2000.0,9.99,0,2014.0,251610,True,2
2,WeiEDKrSat,Counter-Strike,Valve,['Action'],2000.0,9.99,328,2013.0,4000,True,1
3,jarrodtrim,Counter-Strike,Valve,['Action'],2000.0,9.99,337,2012.0,440,True,2
4,UTNerd24,Counter-Strike,Valve,['Action'],2000.0,9.99,5,2014.0,244210,True,1
...,...,...,...,...,...,...,...,...,...,...,...
17623,celebrexISGO,Insurgency,New World Interactive,"['Action', 'Indie', 'Strategy']",2014.0,9.99,0,2015.0,730,True,1
17624,BBiiiirr,Real Horror Stories Ultimate Edition,GameORE,['Indie'],2014.0,1.99,0,2015.0,440,True,1
17625,Hahneex,Infinity Wars: Animated Trading Card Game,Lightmare Studios,"['Free to Play', 'Indie', 'Massively Multiplay...",2014.0,0.00,0,2015.0,730,True,0
17626,76561198222628548,Dungeon Defenders II,Trendy Entertainment,"['Action', 'Free to Play', 'Indie', 'RPG', 'St...",2017.0,0.00,1222,2015.0,370240,False,0


In [149]:
columnas_a_eliminar = ['user_id', 'developer', 'release_date', 'price', 'posted', 'recommend', 'sentiment_analysis']
df_ml = df_ml.drop(columnas_a_eliminar, axis=1)

In [150]:
df_ml.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17628 entries, 0 to 17627
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   title             17628 non-null  object
 1   genres            17628 non-null  object
 2   playtime_forever  17628 non-null  int64 
 3   item_id           17628 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 551.0+ KB


Filtramos el DataFrame unicamente por las columnas que necesitamos, para luego agrupar segun el item_id y la sumatoria de las horas jugadas, ordenamos los valores de mayor a menor y por ultimo filtramos por los primeros 200 items.

In [163]:
df_top100_juegos_jugados = df_ml[['item_id', 'playtime_forever']]
df_top100_juegos_jugados = df_top100_juegos_jugados.groupby('item_id', as_index=False).sum()
df_top100_juegos_jugados = df_top100_juegos_jugados.sort_values(by='playtime_forever', ascending=False)
df_top100_juegos_jugados = df_top100_juegos_jugados.iloc[0:100]
df_top100_juegos_jugados

,item_id,playtime_forever
7,730,8725865
3,440,6397606
10,4000,4996092
90,304930,2026554
73,252490,1890596
...,...,...
51,220200,155356
41,209160,144153
72,248570,141666
83,273110,138780


Convertimos la columna de item_id en una lista

In [174]:
top100_id = df_top100_juegos_jugados['item_id'].to_list()

In [165]:
len(top100_id)

100

Normalizacion de generos
Ahora queremos crear un dataframe para poder comparar la similitud de los items por medio de los generos. Por eso, vamos a realizar el proceso de one-hot enconding para compararlos.

Para eso primero buscamos el listado de generos unicos.

In [166]:
df_generos = df_ml[['genres']]
df_generos['genres'] = df_generos['genres'].apply(eval)
df_generos = df_generos.explode('genres')
df_generos

C:\Users\Coder\AppData\Local\Temp\ipykernel_10904\2583296673.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_generos['genres'] = df_generos['genres'].apply(eval)


,genres
0,Action
2,Action
3,Action
6,Action
7,Action
...,...
17625,Indie
17625,Massively Multiplayer
17625,Strategy
17627,Action


In [167]:
lista_generos = df_generos['genres']
generos_unicos = lista_generos.unique()
generos_unicos

array(['Action', 'Indie', 'Simulation', 'RPG', 'Adventure', 'Strategy',
       'Casual', 'Massively Multiplayer', 'Early Access', 'Free to Play',
       'Racing', 'Sports'], dtype=object)

In [168]:
# Convertimos la cadena en una lista
genres = df_ml['genres'].apply(eval)

for genre in generos_unicos:
    df_ml[genre] = 0  # Inicializamos todas las columnas en 0

for genre in generos_unicos:
    df_ml.loc[genres.apply(lambda x: genre in x), genre] = 1  # Asignamos 1 a las filas donde el género está presente

df_ml.head(2)

,title,genres,playtime_forever,item_id,Action,Indie,Simulation,RPG,Adventure,Strategy,Casual,Massively Multiplayer,Early Access,Free to Play,Racing,Sports
0,Counter-Strike,['Action'],6,1250,1,0,0,0,0,0,0,0,0,0,0,0
2,Counter-Strike,['Action'],328,4000,1,0,0,0,0,0,0,0,0,0,0,0


Filtramos el DataFrame para quedarnos unicamente con el top 200 de juegos mas jugado

In [169]:
df_ml = df_ml[df_ml['item_id'].isin(top100_id)]

In [170]:
#Creamos una tabla pivot con los item_id y las columnas de generos
df_pivot = df_ml.set_index('item_id').drop(['title', 'genres'], axis=1)
df_pivot

,playtime_forever,Action,Indie,Simulation,RPG,Adventure,Strategy,Casual,Massively Multiplayer,Early Access,Free to Play,Racing,Sports
item_id,,,,,,,,,,,,,
1250,6,1,0,0,0,0,0,0,0,0,0,0,0
4000,328,1,0,0,0,0,0,0,0,0,0,0,0
440,337,1,0,0,0,0,0,0,0,0,0,0,0
440,0,1,0,0,0,0,0,0,0,0,0,0,0
1840,332,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,0,1,1,0,0,1,0,0,0,0,1,0,0
730,0,1,1,0,0,0,1,0,0,0,0,0,0
440,0,0,1,0,0,0,0,0,0,0,0,0,0


In [171]:
genero_genero_sim_matrix = pd.DataFrame(cosine_similarity(df_pivot))
genero_genero_sim_matrix

,0,1,2,3,4,5,6,7,8,9,...,10738,10739,10740,10741,10742,10743,10744,10745,10746,10747
0,1.000000,0.986891,0.986877,0.164399,0.986885,0.987678,0.164399,0.995600,0.986620,0.989734,...,0.988297,0.974022,0.986638,0.986531,0.986476,0.082199,0.094916,0.00000,0.00000,0.988569
1,0.986891,1.000000,1.000000,0.003049,1.000000,0.999988,0.003049,0.997671,0.999999,0.999825,...,0.999484,0.961253,0.999997,0.999998,0.999996,0.001524,0.001760,0.00000,0.00000,0.999830
2,0.986877,1.000000,1.000000,0.002967,1.000000,0.999987,0.002967,0.997666,0.999999,0.999824,...,0.999483,0.961240,0.999997,0.999998,0.999996,0.001484,0.001713,0.00000,0.00000,0.999829
3,0.164399,0.003049,0.002967,1.000000,0.003012,0.008000,1.000000,0.071247,0.001381,0.021734,...,0.014917,0.160128,0.001506,0.000834,0.000502,0.500000,0.577350,0.00000,0.00000,0.014490
4,0.986885,1.000000,1.000000,0.003012,1.000000,0.999988,0.003012,0.997669,0.999999,0.999825,...,0.999484,0.961247,0.999997,0.999998,0.999996,0.001506,0.001739,0.00000,0.00000,0.999829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10743,0.082199,0.001524,0.001484,0.500000,0.001506,0.004000,0.500000,0.035624,0.000691,0.010867,...,0.022376,0.240192,0.001506,0.000417,0.000502,1.000000,0.577350,0.50000,0.50000,0.014490
10744,0.094916,0.001760,0.001713,0.577350,0.001739,0.004619,0.577350,0.041135,0.000797,0.012548,...,0.025837,0.184900,0.001739,0.000482,0.000579,0.577350,1.000000,0.57735,0.57735,0.008366
10745,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.014917,0.160128,0.000000,0.000000,0.000502,0.500000,0.577350,1.00000,0.50000,0.000000
10746,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.014917,0.080064,0.000753,0.000000,0.000502,0.500000,0.577350,0.50000,1.00000,0.000000


In [172]:
genero_genero_sim_matrix.columns = df_pivot.index
genero_genero_sim_matrix.set_index(pd.Index(df_pivot.index), inplace=True)

In [142]:
genero_genero_sim_matrix

item_id,1250,251610,4000,440,244210,351640,440,1840,440,219990,...,730,440,246580,255420,261030,730,440,730,370240,730
item_id,,,,,,,,,,,,,,,,,,,,,
1250,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.447214,0.500000,0.000000,0.000000,0.707107,0.577350,0.000000,0.00000,0.447214,0.707107
251610,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.447214,0.500000,0.000000,0.000000,0.707107,0.577350,0.000000,0.00000,0.447214,0.707107
4000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.447214,0.500000,0.000000,0.000000,0.707107,0.577350,0.000000,0.00000,0.447214,0.707107
440,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.447214,0.500000,0.000000,0.000000,0.707107,0.577350,0.000000,0.00000,0.447214,0.707107
244210,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.447214,0.500000,0.000000,0.000000,0.707107,0.577350,0.000000,0.00000,0.447214,0.707107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730,0.577350,0.577350,0.577350,0.577350,0.577350,0.577350,0.577350,0.577350,0.577350,0.577350,...,0.516398,0.577350,0.000000,0.577350,0.408248,1.000000,0.577350,0.57735,0.774597,0.408248
440,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.447214,0.500000,0.000000,0.000000,0.000000,0.577350,1.000000,0.50000,0.447214,0.000000
730,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.447214,0.500000,0.000000,0.500000,0.000000,0.577350,0.500000,1.00000,0.670820,0.000000


In [181]:
df_ml.to_csv(r'C:\Users\Coder\Documents\PI_ML_OPS\Datos\df_ml.csv', index=False, errors='replace', encoding='utf-8')

In [192]:
def recomendacion_juego1(id_juego):

    juegos_similares = genero_genero_sim_matrix.loc[id_juego].sort_values(ascending=False)

    # Obtener el top 5 excluyendo el primero
    top_juegos_similares = juegos_similares.iloc[1:6]  # Cambiado 5 a 6 para obtener el top 5
    lista_de_ids = top_juegos_similares.index.tolist()

    titulos_top = df_ml.loc[df_ml['item_id'].isin(lista_de_ids), 'title']
    lista_de_juegos = [f"{i + 1}- {titulo}" for i, titulo in enumerate(titulos_top.values)]

    return f'Para el ID introducido: {id_juego}, te recomendamos los siguientes juegos similares:\n' + '\n'.join(lista_de_juegos)

In [ ]:
recomendacion_juego1(5)